---
# Feb 1, 2020 Multinomial Logistic Regression with Python Scratch
---
* Name: Jikhan Jeong
* Logistics: All the code and answer are in one Jupyter notebook file
* Data is from 2020 Metric 3 HW1 so I will not upload the dataset due to academic honesty, just see the logic of multinomial logistic regress from scractch 
* Ref: http://learningwithdata.com/logistic-regression-and-optimization.html
* Ref: https://cran.r-project.org/web/packages/mlogit/vignettes/e2nlogit.html
* Ref: Thttps://www.cambridge.org/core/books/discrete-choice-methods-with-simulation/49CABD00F3DDDA088A8FBFAAAD7E9546
---

# 0. Load data and Preparing 
---

In [78]:
import os
os.chdir('/data/cahnrs/jikhan.jeong/metric3_2020')
print(os.getcwd())

/data/cahnrs/jikhan.jeong/metric3_2020


In [79]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import minimize # optimization BFGS
from scipy import stats # to derive p-value

np.set_printoptions(linewidth=np.inf) # For display in one row : ref: https://stackoverflow.com/questions/56204072/numpy-print-matrix-one-row-one-line
np.set_printoptions(threshold=np.inf) # For display in all column : ref: https://stackoverflow.com/questions/1987694/how-to-print-the-full-numpy-array-without-truncation

In [80]:
# Pre-defined function by me for pandas data handling
def unique_value_list(df, var):
    u_list = df[var].unique()
    u_list = sorted(u_list)
    return u_list 

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

def unique_value_distribution(df,var_name): 
    category_distribution = pd.DataFrame(df[var_name].value_counts())
    print_full(category_distribution)

def df_add_var_dummy(df, var):
    df = pd.concat([df, pd.get_dummies(df[var])],axis=1)
    return df 

def select_by_var_value(df, var, value):
    sub_df = df[df[var] == value]
    return sub_df

def sum_df(df):
    print('type: ', type(df))
    print('shape: ', df.shape)

def column_name(df):
    print(len(df.columns.tolist()))
    print(df.columns.tolist())

def count(df):
    return df.count()

def head(df, num=1):
    print('shape: ', df.shape)
    return df.head(num)

def tail(df, num=1):
    print('shape: ',df.shape)
    return df.tail(num)

def re_index(df):
    df = df.reset_index(drop=True)
    return df

# load pandas dataframe or save 
# df.to_csv('df.txt', header = True)
# df = pd.read_csv('df.csv', index_col=[0])

In [81]:
# pandas dataframe to numpy array
# df.values
# Ref: https://stackoverflow.com/questions/13187778/convert-pandas-dataframe-to-numpy-array

In [82]:
df = pd.read_csv('hw1_data.txt', sep='\t') # index_col=[0]

In [83]:
# dataform is wideform
head(df,10)

shape:  (438, 12)


,id,route,toll,median,female,age3050,distance,householdsize,high_income,med_income,occupanc,trans
0,1000169,1,3.250,-4.152752,1,1,34.58,2,1,0,1,1
1,1000187,0,3.250,-4.849391,0,1,26.44,1,0,0,1,0
2,1000201,1,1.625,-4.849391,0,1,24.09,5,0,1,3,1
3,1000209,0,2.900,-3.243314,0,0,26.46,7,0,0,1,0
4,1000282,1,2.900,-2.246180,0,1,45.88,3,0,1,1,1
5,1000412,0,3.250,-5.615552,0,1,37.07,4,0,0,1,0
6,1000438,1,1.450,-1.583941,1,1,26.85,6,0,1,3,1
7,1000445,0,3.250,-4.594528,1,0,33.12,3,0,0,1,0
8,1000468,0,1.650,-0.357715,0,1,43.61,4,0,1,1,0
9,1000479,0,2.900,-1.871945,0,0,33.69,1,0,1,1,1


In [84]:
df.describe()

,id,route,toll,median,female,age3050,distance,householdsize,high_income,med_income,occupanc,trans
count,4.380000e+02,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000
mean,2.559637e+06,0.262557,2.650228,-3.026842,0.319635,0.616438,34.225434,3.529680,0.226027,0.394977,1.376712,0.589041
std,1.401819e+06,0.440527,0.625612,1.505077,0.466868,0.486809,14.185353,1.507505,0.418735,0.489405,0.636140,0.492570
min,1.000169e+06,0.000000,0.825000,-5.628882,0.000000,0.000000,10.180000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,1.005516e+06,0.000000,1.950000,-4.002749,0.000000,0.000000,23.820000,2.000000,0.000000,0.000000,1.000000,0.000000
50%,2.057630e+06,0.000000,2.900000,-3.409326,0.000000,1.000000,31.100000,3.000000,0.000000,0.000000,1.000000,1.000000
75%,4.003429e+06,1.000000,3.000000,-1.465565,1.000000,1.000000,43.022500,4.000000,0.000000,1.000000,2.000000,1.000000
max,5.011583e+06,1.000000,3.250000,-0.134897,1.000000,1.000000,82.050000,10.000000,1.000000,1.000000,3.000000,1.000000


In [85]:
column_name(df)

12
['id', 'route', 'toll', 'median', 'female', 'age3050', 'distance', 'householdsize', 'high_income', 'med_income', 'occupanc', 'trans']


In [86]:
# From pandas dataframe to numpy array
array = df.values

In [87]:
array.shape # shape is attribute, do not use () in the end

(438, 12)

---
# <font color = blue> 1. Simplify the problem
---
* It is not a product level, the no alternative specific price and features are in there. So, I put alternative specific constant to capture unseenable altenative charateristics
* Skip 1. if we can solve 2, then 1 is okay.
* Done problem 2 = **Multinomial Logit** = Dependent **occupanc** : 1 = {solo driving} 2 = {carpool} 3 ={carpool with more people} 
* Not Done problem 3 = **Nested Logit**      = **1st trans** = 1 or 0, **2st opp** = 1 or 0
---

---
# <font color = blue> 2. Problem 2 = Multinomial Logit 
---
* There are no alternative specific features...such as price_choice, choice= 1,2,3. Do it as a practice
* Dependent : **'occupanc'** : 1 = {solo driving} 2 = {carpool} 3 ={carpool with more people} # ignore route to simply the problem
* Indpendent:  1. 'toll', 2. 'median', 3. 'female', 4. 'age3050', 5. 'distance', 6. 'householdsize', 7. 'high_income', 8. 'med_income'
* 2.1 Solving it with long form to easy calculation
* 2.2 Get a utility per each choices
* 2.3 Get an actual and sum of utility
* 2.4 Get a probability
* 2.5 Get a negative loglikelihood
* Optimize and get a parameters
* Compare it with stata output
---

In [88]:
#  (438, 11)
df2 = df[['id', 'occupanc', 'route', 'toll', 'median', 'female', 'age3050', 'distance', 'householdsize', 'high_income', 'med_income']]
sum_df(df2)

type:  <class 'pandas.core.frame.DataFrame'>
shape:  (438, 11)


In [89]:
# Alternative Specific constant added : not exlcuding 1 term underconsideration of DF, we capture all variance by excluding base group
df2 = df_add_var_dummy(df2, 'occupanc')
df2

,id,occupanc,route,toll,median,female,age3050,distance,householdsize,high_income,med_income,1,2,3
0,1000169,1,1,3.250,-4.152752,1,1,34.58,2,1,0,1,0,0
1,1000187,1,0,3.250,-4.849391,0,1,26.44,1,0,0,1,0,0
2,1000201,3,1,1.625,-4.849391,0,1,24.09,5,0,1,0,0,1
3,1000209,1,0,2.900,-3.243314,0,0,26.46,7,0,0,1,0,0
4,1000282,1,1,2.900,-2.246180,0,1,45.88,3,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,5004374,3,0,1.450,-2.249893,0,1,27.97,6,0,0,0,0,1
434,5006382,1,0,1.950,-1.206373,1,0,33.47,5,1,0,1,0,0
435,5006513,1,0,2.900,-2.595311,0,1,44.56,4,0,0,1,0,0
436,5006568,1,0,2.900,-2.422315,0,0,61.02,1,0,0,1,0,0


In [90]:
df2.to_csv('df_Q2_wide_mlogit.csv', header = True)

### Pandas Value as a numpy value 

In [91]:
data2 = df2.values

In [92]:
dev = data2[:, -3:]   # (438, 3) 1 dependent = occupanc, three possible choices {choice 1 : sole, choice 2 : carpool , choice 3: carpool_2} eg. array([1., 0., 0.])
inv = data2[:, 2:-3]  # (438, 9) 9 independents including attribute specific constants (438, 9) # array([ 1.      ,  3.25    , -4.152752,  1.      ,  1.      , 34.58    ,  2.      ,  1.      ,  0.      ,  1.      ,  0.      ,  0.      ])

In [122]:
# coefficient for independent variables including constant values
coe = 0.001*np.ones(11).T # 11 = 9 independent + 2 constants for alternative specific constant (ASC) exlcuding base group = carpool_2 due to degree of freedom

In [94]:
num_of_sample = dev.shape[0]    
constant = np.ones((num_of_sample, 1)) # (438, 1)

In [95]:
dev1_X = np.hstack((constant,inv))  # dev1.shape (438, 10)
dev2_X = np.hstack((constant,inv))  # dev2.shape (438, 10)
dev3_X = inv                        # dev3.shape (438,  9) # no constant = base group 
# numpy.hstack(tup) : stack arrays in sequence horizontally (column wise).   

In [96]:
coe1 = np.hstack((coe[0], coe[2:])).reshape(10,1) # coe1.shape (10,1)
coe2 = np.hstack((coe[1], coe[2:])).reshape(10,1) # coe2.shape (10,1)
coe3 = coe[2:].reshape(9,1)                      # coe3.shape (9,1)

---
### Derive Probability 
* Step1: Get V(Y = {1,2,3}| U = V + error, V = X*coefficients)
* Step2: Get Exp(V)
* Step3: Get Sum(Exp(V)) -- Denominators
* Step4: Get Probability(Y) = Exp(V(Y=i)) / Sum(Exp(V(Y=i)))
--- 

#### Step1: Get V(Y = {1,2,3}| U = V + error, V = X*coefficients)

In [97]:
observed_utility_choice1 = np.dot(dev1_X, coe1)
observed_utility_choice2 = np.dot(dev2_X, coe2)
observed_utility_choice3 = np.dot(dev3_X, coe3)

#### Step2: Get Exp(V)

In [98]:
exp_v1 =  np.exp(observed_utility_choice1)
exp_v2 =  np.exp(observed_utility_choice2)
exp_v3 =  np.exp(observed_utility_choice3)

#### Step3: Get Sum(Exp(V)) -- Denominators

In [99]:
exp_v_sum = exp_v1 + exp_v2 + exp_v3  # Denominator (438, 1)

#### Step4: Get Probability(Y) = Exp(V(Y=i)) / Sum(Exp(V(Y=i)))

In [100]:
p_dev1 = exp_v1/exp_v_sum # (438, 1)
p_dev2 = exp_v2/exp_v_sum
p_dev3 = exp_v3/exp_v_sum

In [101]:
likelihood = np.multiply(p_dev1, dev[:,0].reshape(438,1))  + np.multiply(p_dev2, dev[:,1].reshape(438,1)) + np.multiply(p_dev3, dev[:,2].reshape(438,1)) # (438, 1)
                                                                               
# np.multiply: Multiply arguments element-wise.
# Ref: https://docs.scipy.org/doc/numpy/reference/generated/numpy.multiply.html

In [102]:
# nll: engative log likelihood to optimize it with minimization
# likelihood function itself, negative definite = concave function
negative_log_likelihood = -np.sum(np.log(likelihood))

---
# Make a function for minimization : using scipy.optimize
---
* no gradient information: minimize(f, x0, method='bfgs'). bfgs giving an inverse hessian matrix to get se, t-value, and p-values, no requires for jacobian 
* with gradient, you can either have a callable which returns the function and the gradient: minimize(fnc_and_jac, x0, method='bfgs', jac=True), fnc_and_jac in here defines jacobian (inside of function)
* set jac to a callable, which should have the same signature as the cost function and return the gradient: minimize(f, x0, method='bfgs', jac=jac). making anddition function jac for jacobain (outside of f function) 
* Ref: https://stackoverflow.com/questions/29324222/how-can-i-do-a-maximum-likelihood-regression-using-scipy-optimize-minimize
* Ref: https://stackoverflow.com/questions/27259842/logistic-regression-and-scipy-optimization-with-fmin-bfgs
---

In [103]:
from scipy.optimize import minimize # optimization BFGS

In [116]:
def nll(coe): # dev = dev(438,3) inv = inv (438,9), coe = coe ( 11 parameters  = 2 constant + 9 variables )
    
    num_of_sample = dev.shape[0]                        #  438
    constant = np.ones((num_of_sample, 1))              # (438, 1)
    
    dev1_X = np.hstack((constant,inv))                  # dev1.shape (438, 10)
    dev2_X = np.hstack((constant,inv))                  # dev2.shape (438, 10)
    dev3_X = inv                                        # dev3.shape (438,  9) # no constant = base group 
                                                        # numpy.hstack(tup) : stack arrays in sequence horizontally (column wise). 
    
    coe1 = np.hstack((coe[0], coe[2:])).reshape(inv.shape[1]+1,1)   # coe1.shape (10,1)
    coe2 = np.hstack((coe[1], coe[2:])).reshape(inv.shape[1]+1,1)   # coe2.shape (10,1)
    coe3 = coe[2:].reshape(inv.shape[1],1)                          # coe3.shape (9,1)
    
    observed_utility_choice1 = np.dot(dev1_X, coe1)
    observed_utility_choice2 = np.dot(dev2_X, coe2)
    observed_utility_choice3 = np.dot(dev3_X, coe3)
    
    exp_v1 =  np.exp(observed_utility_choice1)
    exp_v2 =  np.exp(observed_utility_choice2)
    exp_v3 =  np.exp(observed_utility_choice3)
    
    exp_v_sum = exp_v1 + exp_v2 + exp_v3                             # Denominator (438, 1)
    
    p_dev1 = exp_v1/exp_v_sum                                        # (438, 1)
    p_dev2 = exp_v2/exp_v_sum
    p_dev3 = exp_v3/exp_v_sum
    
    likelihood = np.multiply(p_dev1, dev[:,0].reshape(num_of_sample,1))  + np.multiply(p_dev2, dev[:,1].reshape(num_of_sample,1)) + np.multiply(p_dev3, dev[:,2].reshape(num_of_sample,1))
    
    negative_log_likelihood = -np.sum(np.log(likelihood))
    
    return negative_log_likelihood

In [152]:
optimization_output = minimize(nll, x0=coe, method='bfgs')
print(optimization_output)

      fun: 341.5822926654646
 hess_inv: array([[ 2.98725454e-02,  2.64220305e-02, -1.80146034e-06, -6.88516518e-04, -6.88516518e-04,  0.00000000e+00, -1.84565544e-06, -6.88516518e-04, -6.88516518e-04, -6.88472322e-04,  0.00000000e+00],
       [ 2.64220305e-02,  3.70333321e-02, -1.47027777e-06, -2.41094255e-04, -2.41094255e-04,  0.00000000e+00, -1.48158340e-06, -2.41094255e-04, -2.41094255e-04, -2.41082950e-04,  0.00000000e+00],
       [-1.80146034e-06, -1.47027777e-06,  9.99999996e-01,  4.02573243e-08,  4.02573243e-08,  0.00000000e+00, -4.46605876e-09,  4.02573243e-08,  4.02573243e-08,  4.02544087e-08,  0.00000000e+00],
       [-6.88516518e-04, -2.41094255e-04,  4.02573243e-08,  1.00000009e+00,  8.85110634e-08,  0.00000000e+00,  4.20239677e-08,  8.85110634e-08,  8.85110634e-08,  8.67444199e-08,  0.00000000e+00],
       [-6.88516518e-04, -2.41094255e-04,  4.02573243e-08,  8.85110634e-08,  1.00000009e+00,  0.00000000e+00,  4.20239677e-08,  8.85110634e-08,  8.85110634e-08,  8.67444199e-08

In [153]:
print('coefficient :\n', optimization_output.x)

coefficient :
 [2.12565424e+00 8.99941439e-01 9.96045108e-04 9.90874696e-04 9.90874696e-04 1.00000000e-03 9.94723011e-04 9.90874696e-04 9.90874696e-04 9.92196793e-04 1.00000000e-03]


In [154]:
print('inverse hessian matrix :\n', optimization_output.hess_inv)

inverse hessian matrix :
 [[ 2.98725454e-02  2.64220305e-02 -1.80146034e-06 -6.88516518e-04 -6.88516518e-04  0.00000000e+00 -1.84565544e-06 -6.88516518e-04 -6.88516518e-04 -6.88472322e-04  0.00000000e+00]
 [ 2.64220305e-02  3.70333321e-02 -1.47027777e-06 -2.41094255e-04 -2.41094255e-04  0.00000000e+00 -1.48158340e-06 -2.41094255e-04 -2.41094255e-04 -2.41082950e-04  0.00000000e+00]
 [-1.80146034e-06 -1.47027777e-06  9.99999996e-01  4.02573243e-08  4.02573243e-08  0.00000000e+00 -4.46605876e-09  4.02573243e-08  4.02573243e-08  4.02544087e-08  0.00000000e+00]
 [-6.88516518e-04 -2.41094255e-04  4.02573243e-08  1.00000009e+00  8.85110634e-08  0.00000000e+00  4.20239677e-08  8.85110634e-08  8.85110634e-08  8.67444199e-08  0.00000000e+00]
 [-6.88516518e-04 -2.41094255e-04  4.02573243e-08  8.85110634e-08  1.00000009e+00  0.00000000e+00  4.20239677e-08  8.85110634e-08  8.85110634e-08  8.67444199e-08  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+

In [155]:
print('stand error of coefficient :\n', np.sqrt(optimization_output.hess_inv.diagonal()))

stand error of coefficient :
 [0.17283676 0.19244046 1.         1.00000004 1.00000004 1.         1.         1.00000004 1.00000004 1.00000004 1.        ]


In [156]:
print('t-value :\n', optimization_output.x / np.sqrt(optimization_output.hess_inv.diagonal()))

t-value :
 [1.22986235e+01 4.67646680e+00 9.96045111e-04 9.90874652e-04 9.90874652e-04 1.00000000e-03 9.94723013e-04 9.90874652e-04 9.90874652e-04 9.92196751e-04 1.00000000e-03]


In [157]:
from scipy import stats

tt = optimization_output.x / np.sqrt(optimization_output.hess_inv.diagonal())
p_value_list = []
for i in range(len(tt)):
    pval = stats.t.sf(np.abs(tt[i]), num_of_sample-1)*2  # two-sided pvalue = Prob(abs(t)>tt)
    pval = round(pval, 4)
    p_value_list.append(pval)
p_values = np.asarray(p_value_list, dtype=np.float32)
print('pvalue: ', p_value_list)

pvalue:  [0.0, 0.0, 0.9992, 0.9992, 0.9992, 0.9992, 0.9992, 0.9992, 0.9992, 0.9992, 0.9992]


In [158]:
table =[]
table.append(optimization_output.x)
table.append(np.sqrt(optimization_output.hess_inv.diagonal()))
table.append(optimization_output.x / np.sqrt(optimization_output.hess_inv.diagonal()))
table.append(p_values)
table

[array([2.12565424e+00, 8.99941439e-01, 9.96045108e-04, 9.90874696e-04, 9.90874696e-04, 1.00000000e-03, 9.94723011e-04, 9.90874696e-04, 9.90874696e-04, 9.92196793e-04, 1.00000000e-03]),
 array([0.17283676, 0.19244046, 1.        , 1.00000004, 1.00000004, 1.        , 1.        , 1.00000004, 1.00000004, 1.00000004, 1.        ]),
 array([1.22986235e+01, 4.67646680e+00, 9.96045111e-04, 9.90874652e-04, 9.90874652e-04, 1.00000000e-03, 9.94723013e-04, 9.90874652e-04, 9.90874652e-04, 9.92196751e-04, 1.00000000e-03]),
 array([0.    , 0.    , 0.9992, 0.9992, 0.9992, 0.9992, 0.9992, 0.9992, 0.9992, 0.9992, 0.9992], dtype=float32)]

In [159]:
mlogit_output_df = pd.DataFrame(table, columns = ['constant1' , 'constant2', 'route', 'toll', 'median', 'female', 'age3050', 'distance', 'householdsize', 'high_income', 'med_income'], index=['coefficient', 'standard error ', 't-value' , 'p-value'])
mlogit_output_df

,constant1,constant2,route,toll,median,female,age3050,distance,householdsize,high_income,med_income
coefficient,2.125654,0.899941,0.000996,0.000991,0.000991,0.0010,0.000995,0.000991,0.000991,0.000992,0.0010
standard error,0.172837,0.192440,1.000000,1.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.0000
t-value,12.298624,4.676467,0.000996,0.000991,0.000991,0.0010,0.000995,0.000991,0.000991,0.000992,0.0010
p-value,0.000000,0.000000,0.999200,0.999200,0.999200,0.9992,0.999200,0.999200,0.999200,0.999200,0.9992


---
# Interpretation

* Only ASC is significant, it may not converge, coding is reasonable but model should be redefined but this is homework so it is okay
* It should put something more alternative specific feature variables, even though the same alternative the toll(=financial cost) are not following the variation of 
  alternative changes. 
* It may better to apply this model with alternative specific price and feature can be comparable such as food1, food2, food3. or car1, car2. car3. 
---